<a href="https://colab.research.google.com/github/lgy112112/FUNet/blob/master/FUNet_Example_Use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [28]:
import torch.nn as nn

def unet_code_generator(depth, n_channels, n_classes, bilinear=False):
    """
    Generates the Python code text for a UNet architecture with specified depth.

    Args:
        depth (int): The desired depth of the UNet model.
        n_channels (int): Number of input channels.
        n_classes (int): Number of output classes.
        bilinear (bool): Whether to use bilinear upsampling (default: False).

    Returns:
        str: The generated Python code text defining the UNet class.
    """

    code_text = ""

    # --- Beginning (remains the same) ---
    code_text += "class UNet(nn.Module):\n"
    code_text += f"    def __init__(self, n_channels={n_channels}, n_classes={n_classes}, bilinear={bilinear}):\n"
    code_text += "        super(UNet, self).__init__()\n"
    code_text += "        self.n_channels = n_channels\n"
    code_text += "        self.n_classes = n_classes\n"
    code_text += "        self.bilinear = bilinear\n"
    code_text += "        factor = 2 if bilinear else 1\n\n"

    # --- Encoder blocks ---
    code_text += "        self.inc = DoubleConv(n_channels, 64)\n"  # Initial convolution
    for i in range(1, depth):
        in_channels = 64 * 2**(i-1)
        out_channels = in_channels * 2
        divide = " // factor" if i == depth - 1 else ""  # Apply division only on last 'down'
        code_text += f"        self.down{i} = Down({in_channels}, {out_channels}{divide})\n"
    code_text += "\n"
    record = out_channels
# --- Decoder blocks ---
    for i in range(depth-1, 0, -1):
        in_channels = out_channels  # Corrected calculation
        out_channels = in_channels // 2
        divide = " // factor" if i != 1 else ""
        code_text += f"        self.up{i} = Up({in_channels}, {out_channels}{divide}, bilinear)\n"
    code_text += "        self.outc = OutConv(64, n_classes)\n\n"


    # --- Forward function ---
    code_text += "    def forward(self, x):\n"
    code_text += "        x1 = self.inc(x)\n"

    for i in range(1, depth):
        code_text += f"        x{i+1} = self.down{i}(x{i})\n"

    # Corrected order for upsampling
    for i in range(depth-1, 0, -1): # (4,0,-1)
        if i == depth-1:
            code_text += f"        x = self.up{i}(x{depth}, x{depth-1})\n"
        else:
            code_text += f"        x = self.up{i}(x, x{i})\n"

    code_text += "        logits = self.outc(x)\n"
    code_text += "        return logits\n"

    return code_text


In [ ]:
code_text = unet_code_generator(depth=5, n_channels=3, n_classes=10, bilinear=False)
print(code_text)

In [37]:
class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=10, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        factor = 2 if bilinear else 1

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        self.down5 = Down(1024, 2048 // factor)

        self.up5 = Up(2048, 1024 // factor, bilinear)
        self.up4 = Up(1024, 512 // factor, bilinear)
        self.up3 = Up(512, 256 // factor, bilinear)
        self.up2 = Up(256, 128 // factor, bilinear)
        self.up1 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x6 = self.down5(x5)
        x = self.up5(x6, x5)
        x = self.up4(x, x4)
        x = self.up3(x, x3)
        x = self.up2(x, x2)
        x = self.up1(x, x1)
        logits = self.outc(x)
        return logits

In [38]:
model = UNet(n_channels=3, n_classes=10, bilinear=False)
test_input = torch.randn(1, 3, 256, 256)
output = model(test_input)
print(output.shape)

torch.Size([1, 10, 256, 256])
